#### GISC 420 T1 2022
# Dictionaries
This notebook is an overview of material in Chapter 11 of [Think Python](https://greenteapress.com/wp/think-python-2e/).

The `dict` or *dictionary* type is a very useful kind of collection in Python, which stores *values* indexed by *keys*. It is a *mapping* from the keys to the values.

An empty dictionary can be made in one of two ways either by invoking `dict()` or with empty 'curly' brackets `{}`

In [ ]:
eng_tereo = dict()
tereo_eng = {}

We can add items to an empty dictionary like this

In [ ]:
eng_tereo['hello'] = 'kia ora'
tereo_eng['kia ora'] = 'hello'

The values in the square brackets here are the *keys*, while the values assigned are the *values* associated with those keys. We can retrieve them like this:

In [ ]:
print(eng_tereo['hello'])

### A brief digression
Add another word

In [ ]:
eng_tereo['world'] = 'ao tūroa'

Make a little function

In [ ]:
def speak_tereo(words):
    tereo_words = []
    for word in words.split():
        tereo_words.append(eng_tereo[word])
    print(' '.join(tereo_words))

And now we can do this

In [ ]:
speak_tereo("hello world")

\[Translation is obviously way more complicated than simple word substitution, but I couldn't resist...\]

## OK... back in the room
In dictionaries, keys and values can be almost anything, provided the keys are *immutable*. We've already seen that lists are mutable, so they can't be used as keys, but _tuples_ can. A tuple is like a list, but has round brackets:

In [ ]:
values = (1, 2, 3, 4)
values

Strings are also fine as dictionary keys, as are integers and floats, although you have to be careful about floats, because two values that seem the same might not be _exactly_ the same. 

Regardless, the fact you can use more or less any value as a key in a dictionary makes dictionaries incredibly useful data structures.

### Some basics
We make an empty dictionary with curly brackets, or with the `dict()` function. We can initialise it with some key-value pairs like this

In [ ]:
my_dict = {"one": 1, "two": 2, "three": 3}

We can initialise a dictionary from two lists or two tuples (or any pair of iterable objects of the same lengths) using the `zip()` function, as shown below.

In [ ]:
word = list("dictionary")
indexes = range(len(word))
print(indexes, word)

In [ ]:
letters = dict(zip(indexes, word))
letters

Slightly confusingly, iterating over a dictionary with a for loop will return the _keys_ (not the values):

In [ ]:
for k in letters:
    print(k)

If you want the values, use `.values()`; if you want the key-value pairs use `.items()`:

In [ ]:
for v in letters.values():
    print(v)

In [ ]:
for k, v in letters.items():
    print(f"value at key {k} is {v}")

### Inverting a dictionary (advanced)
Sometimes you want to invert a dictionary so you can do lookups in the opposite direction. This can get tricky if there are duplicate values, as below:

In [ ]:
letters_rev = {}
for k, v in letters.items():
    letters_rev[v] = k
letters_rev

Notice how the letter `i` which appears twice in the values appears only once in the inverted dictionary, because they second time it shows up overwrites the first time. There is a way to work around this with something called a `defaultdict` from the `collections` module. This special type of dictionary can have a default value specified. If we specify a list as the default type, then we can `append` to the list at each key position any duplicate entries.

In [ ]:
from collections import defaultdict
letters_rev = defaultdict(list)
for k, v in letters.items():
    letters_rev[v].append(k)
dict(letters_rev)

## Using a dictionary to count things
In addition to the 'translation' kind of functionality we've just seen they can be used to count things.

For example, make a list of 1000 random numbers (although to save space, just look at the first 20 values). We can do this with the `random.randint()` function.

In [ ]:
import random
numbers = []
for i in range(10000):
    numbers.append(random.randint(0, 50))
numbers[:20]

How could we count the number of occurrences of each possible value from 0 to 50 in this list? One way would be to make a new list, like this

In [ ]:
counts = [0] * 51 # we need 51 because 0..50 is 51 values!
counts[:20]

Then we can go through the list of numbers like this

In [ ]:
for n in numbers:
    counts[n] = counts[n] + 1

And now print it

In [ ]:
# Note using an enumerate here
for number, count in enumerate(counts):
    print(f"{count} {number}'s in the list")

That works OK, but what if the numbers were in a different range? Say they were not 0 to 50, but 100 to 150? Or 10001 to 10020? We would have to know in advance something about what is in the list of numbers to set up our list of counting bins appropriately and change the code accordingly. You can give it a try below

In [ ]:
numbers = []
for i in range(10000):
    numbers.append(random.randint(10001, 10020))

# write some code to assemble these counts into a list...

Not so easy... and in fact... a dictionary does exactly what we need! 

With a dictionary it is straightforward.

In [ ]:
counts = {}
for n in numbers:
    if n in counts:
        counts[n] = counts[n] + 1
    else:
        counts[n] = 1
        
counts

The only issue with this approach is that the counts are not ordered, but that is easy to fix as we have done below with a `sorted` function call (and it might not even matter that much). 

In [ ]:
for number, count in sorted(counts.items()):
    print(f"{count} {number}'s in the list")

The key difference in using the dictionary, is that it only makes a counter for a particular number, *if it is needed*.

Also... because a dictionary can use more or less anything as a key, we can use exactly the same code to count the letters in a word or sentence.

In [ ]:
def count_occurrences(sequence):
    counts = {}
    for s in sequence:
        if s in counts:
            counts[s] = counts[s] + 1
        else:
            counts[s] = 1
    return counts

count_occurrences("Dictionaries are cool")

In fact, the above function is so flexible it can be used to count numbers in a list as well! (Try it.)

#### Slightly more advanced
We can make this function even cleaner with a `defaultdict` with the default type set to be an integer.

In [ ]:
def count_occurences(sequence):
    counts = defaultdict(int)
    for s in sequence:
        counts[s] = counts[s] + 1
    return counts

dice_rolls = []
for i in range(1000):
    dice_rolls.append(random.randint(1, 6))
sorted(count_occurences(dice_rolls).items())

## The many uses of dictionaries
We've already seen two uses for dictionaries: translations (mappings) and counting things. Dictionaries are a tremendously useful *data structure* that can be used in all kinds of situations. The chapter in _Think Python_ discusses _memoizing_ in some detail. It is worth taking a look at this closely. I have replicated the essentials below.

First, we make a *recursive* function to calculate *Fibonacci* numbers. These are numbers in the series, 1, 1, 2, 3, 5, 8, 13, ... and so on, where the next number in the series each time is the sum of the previous two and we set things running starting with 1, 1.

In [ ]:
# Recursive function to calculate Fibonacci numbers
def fib(n, debug = False):
    # if input is 0, result is 0
    if n == 0:
        if debug: 
            print("getting fib(0)")
        return 0
    # if input is 1 result is 1
    if n == 1:
        if debug: 
            print("getting fib(1)")
        return 1
    # otherwise the result is the sum of 
    # the previous two entries
    return fib(n - 1, debug) + fib(n - 2, debug)


This works OK for fairly small values of `n`

In [ ]:
fib(5, True)

But there is a problem, which you can see because of the *debugging code* I put in to tell us every time we are calculating a result for `fib(0)` or `fib(1)`. The problem is that we are repeatedly getting results we already know.

You can see how much of a problem this is by running the cell below. After you have run it, I recommend commenting the code below out, so you don't accidentally run it again...

In [ ]:
fib(20)

Using a dictionary, we can make this code *much more efficient* by storing intermediate values as we go and using those if we already know them.  Here's a better Fibonacci function.

In [ ]:
# Recursive function to calculate Fibonacci numbers
# using a dictionary to remember known results

known_fib = {0:0, 1:1}
def fib2(n, debug = False):
    if (n == 0 or n == 1) and debug: 
        print('getting fib(' + str(n) + ')')
    if n in known_fib:
        return known_fib[n]
    # otherwise the result is the sum of 
    # the previous two entries
    result = fib2(n - 1, debug) + fib2(n - 2, debug)
    # and we store it in the dictionary of known results
    known_fib[n] = result
    return result

fib2(500, debug = True)

\[You don't even want to think about how long the first version of this function would take to run for an input of 500... **don't** try it!\]


This is a good example of how we need to use program code **in conjunction with appropriate and well chosen data structures** to efficiently solve programming problems, and is something we will try to look at it in the next couple of weeks. An ability to do this is partly experience, but it is also something that can be learned. An alternative is to use well designed modules written by others to solve problems which is what gets us into APIs, another topic for the coming weeks.

### By the way...
It is worth knowing that there is a module (non-standard, so it has to be installed) for handling memoizing, called `cachetools`. You can find out more about it [here](https://pypi.org/project/cachetools/).

In [ ]:
pip install cachetools

In [ ]:
from cachetools import cached

@cached(cache={})
def fib3(n):
    return n if n < 2 else fib3(n - 1) + fib3(n - 2) # Note careful here to call fib3!

fib3(500)